# 本次研讨课内容:
   问答摘要内容讲解:
    1. vocab对象构建
    2. batcher 方法构建
    3. PGN Model.
    4. coverage
    5. coverage_loss

In [4]:
import tensorflow as tf

![](img/Text.png)

蓝色的字体表示的是参考摘要，三个模型的生成摘要的结果差别挺大。红色字体表明了不准确的摘要细节生成(UNK未登录词，无法解决OOV问题)，绿色的字体表明了模型生成了重复文本。

![](img/rouge.png)

![](img/aistudio.png)

# 1.PGN

1. sequence-to-sequence mode baseline,
2. pointer generater mode 
3. coverage机制

# Seq2Seq
![](img/seq2seq.png)

-----

# PGN
![](img/pgn.png)

## 0. vocab对象构建

In [46]:
class Vocab:
    PAD_TOKEN = '<PAD>'
    UNKNOWN_TOKEN = '<UNK>'
    START_DECODING = '<START>'
    STOP_DECODING = '<STOP>'

    def __init__(self, vocab_file, vocab_max_size=None):
        """
        Vocab 对象,vocab基本操作封装
        :param vocab_file: Vocab 存储路径
        :param vocab_max_size: 最大字典数量
        """
        self.word2id, self.id2word = self.load_vocab(vocab_file, vocab_max_size)
        self.count = len(self.word2id)

    @staticmethod
    def load_vocab(file_path, vocab_max_size=None):
        """
        读取字典
        :param file_path: 文件路径
        :return: 返回读取后的字典
        """
        vocab = {}
        reverse_vocab = {}
        for line in open(file_path, "r", encoding='utf-8').readlines():
            word, index = line.strip().split("\t")
            index = int(index)
            # 如果vocab 超过了指定大小
            # 跳出循环 截断
            if vocab_max_size and index > vocab_max_size:
                print("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (
                    vocab_max_size, index))
                break
            vocab[word] = index
            reverse_vocab[index] = word
        return vocab, reverse_vocab

    def word_to_id(self, word):
        if word not in self.word2id:
            return self.word2id[self.UNKNOWN_TOKEN]
        return self.word2id[word]

    def id_to_word(self, word_id):
        if word_id not in self.id2word:
            raise ValueError('Id not found in vocab: %d' % word_id)
        return self.id2word[word_id]

    def size(self):
        return self.count

## Q 1. oov词去哪里取?

# 1. batcher 改进

### predictions -> idx2word -> `QQ224,这是车辆<UNK><UNK>`

### enc_extended_inp -> idx2word ->  `道奇,锋哲,昨晚看到一台车车牌是绿色的？这是什么牌？`

### enc_inp -> idx2word->  `<UNK>,<UNK>,昨晚看到一台车车牌是绿色的？这是什么牌？`

### article_oovs ->` [道奇 , 锋哲]`

### dec_input -> idx2word-> `<start>QQ224,这是车辆<UNK><UNK><end><pad><pad>`

### target -> idx2word-> ` QQ224,这是车辆道奇,锋哲<end><pad>`

In [ ]:
dataset = tf.data.Dataset.from_generator(
        lambda: generator(params, vocab, max_enc_len, max_dec_len, mode, batch_size),
        output_types={
            "enc_len": tf.int32,
            "enc_input": tf.int32,
            "enc_extended_inp": tf.int32,
            
            "article_oovs": tf.string,
            "dec_input": tf.int32,
            "target": tf.int32,
            "dec_len": tf.int32,
            "article": tf.string,
            "abstract": tf.string,
            "abstract_sents": tf.string,
            "sample_decoder_pad_mask": tf.int32,
            "sample_encoder_pad_mask": tf.int32,
        },
        output_shapes={
            "enc_len": [],
            "enc_input": [None],
            "enc_extended_inp": [None],
            
            "article_oovs": [None],
            "dec_input": [None],
            "target": [None],
            "dec_len": [],
            "article": [],
            "abstract": [],
            "abstract_sents": [],
            "sample_decoder_pad_mask": [None],
            "sample_encoder_pad_mask": [None]
        })

In [ ]:
dataset = dataset.padded_batch(batch_size,
               padded_shapes=({"enc_len": [],
                               "enc_input": [None],
                               "enc_input_extend_vocab": [None],
                               "article_oovs": [None],
                               "dec_input": [max_dec_len],
                               "target": [max_dec_len],
                               "dec_len": [],
                               "article": [],
                               "abstract": [],
                               }),
               padding_values={"enc_len": -1,
                               "enc_input": vocab.word2id[Vocab.PAD_TOKEN],
                               "enc_input_extend_vocab": vocab.word2id[Vocab.PAD_TOKEN],
                               "article_oovs": b'',
                               "dec_input": vocab.word2id[Vocab.PAD_TOKEN],
                               "target": vocab.word2id[Vocab.PAD_TOKEN],
                               "dec_len": -1,
                               "article": b"",
                               "abstract": b"",
                               },
               drop_remainder=True)

# PGN Model

## Ppgn  

### seq2seq+point

混合了 Baseline seq2seq和PointerNetwork的网络，它具有Baseline seq2seq的生成能力和PointerNetwork的Copy能力。如何权衡一个词应该是生成的还是复制的？原文中引入了一个权重$p_{gen}$

从Baseline seq2seq的模型结构中得到了$S_t$和$h^*_t$，和解码器输入 $x_t$ 一起来计算 $p_{gen}$ ： 

![](readme/pgn1.png)

# Pgen ∈ [0,1]

+ context vector $h^*_t$
+ decoder input $x_t$ 
+ the decoder state $S_t$

In [ ]:
class Pointer(tf.keras.layers.Layer):

    def __init__(self):
        super(Pointer, self).__init__()
        self.w_s_reduce = tf.keras.layers.Dense(1)
        self.w_i_reduce = tf.keras.layers.Dense(1)
        self.w_c_reduce = tf.keras.layers.Dense(1)

    def call(self, context_vector, dec_hidden, dec_inp):
        return tf.nn.sigmoid(self.w_s_reduce(dec_hidden) + self.w_c_reduce(context_vector) + self.w_i_reduce(dec_inp))

In [41]:
p_gens = []
for t in range(dec_target.shape[1]):
    .....
    p_gen = self.pointer(context_vector, dec_hidden, dec_x)

p_gens.append(p_gen)

![](readme/pgn.png)

# Final_dists

这时，会扩充单词表形成一个更大的单词表--扩充单词表(将原文当中的单词也加入到其中)，该时间步的预测词概率为：

![](readme/pw.png)

其中 $a_i^t$ 表示的是原文档中的词。我们可以看到解码器一个词的输出概率有其是否拷贝是否生成的概率和决定。当一个词不出现在常规的单词表上时$P_{vocab}(w)$ 为0.

![](readme/pgn.png)

## predictions -> idx2word -> `QQ224,这是车辆<UNK><UNK>`

## enc_extended_inp -> idx2word ->  `道奇,锋哲,昨晚看到一台车车牌是绿色的？这是什么牌？`

## enc_inp -> idx2word->  `<UNK>,<UNK>,昨晚看到一台车车牌是绿色的？这是什么牌？`

In [ ]:
final_dists=calc_final_dist(enc_extended_inp,# 原始输入
                             predictions,# 原始预测概率
                             attentions, # att权重
                             p_gens, # pgn概率
                             batch_oov_len,# 2
                             self.params["vocab_size"],# 原始的wocab size
                             self.params["batch_size"])

vocab -> `{
    <UNK>:0,
    昨晚:1,
    看到:2,
    一台车:3,
    车牌:4,
    是绿色的:5,
    ？:6,
    这是什么:7,
    牌:8
}`


extended_vsize=10

extend_vocab ->
`{
    <UNK>:0,
    昨晚:1,
    看到:2,
    一台车:3,
    车牌:4,
    是绿色的:5,
    ？:6,
    这是什么:7,
    牌:8,
    道奇:9,
    锋哲:10.
}`

### predictions -> idx2word -> `QQ224,这是车辆<UNK><UNK>`

### enc_extended_inp -> idx2word ->  `道奇,锋哲,昨晚看到一台车车牌是绿色的？这是什么牌？`

### enc_inp -> idx2word->  `<UNK>,<UNK>,昨晚看到一台车车牌是绿色的？这是什么牌？`

### article_oovs ->` [道奇 , 锋哲]`

### dec_input -> idx2word-> `<start>QQ224,这是车辆<UNK><UNK><end><pad><pad>`

### target -> idx2word-> ` QQ224,这是车辆道奇,锋哲<end><pad>`

![](readme/pgn.png)

enc_inp <unk> <unk> 1 2 3 4 5 6 7 8

enc_extended_inp 9 10 1 2 3 4 5 6 7 8

In [ ]:
final_dists=calc_final_dist(enc_extended_inp,# 原始输入
                             predictions,# 原始预测概率
                             attentions, # att权重
                             p_gens, # pgn概率
                             batch_oov_len,# 2
                             self.params["vocab_size"],# 原始的wocab size
                             self.params["batch_size"])

![](readme/pw.png)

In [ ]:
def calc_final_dist(_enc_batch_extend_vocab, vocab_dists, attn_dists, p_gens, batch_oov_len, vocab_size, batch_size):
    """
    Calculate the final distribution, for the pointer-generator model
    Args:
    vocab_dists: The vocabulary distributions. List length max_dec_steps of (batch_size, vsize) arrays.
                The words are in the order they appear in the vocabulary file.
    attn_dists: The attention distributions. List length max_dec_steps of (batch_size, attn_len) arrays
    Returns:
    final_dists: The final distributions. List length max_dec_steps of (batch_size, extended_vsize) arrays.
    """
    # Multiply vocab dists by p_gen and attention dists by (1-p_gen)
    vocab_dists = [p_gen * dist for (p_gen, dist) in zip(p_gens, vocab_dists)]
    
    attn_dists = [(1 - p_gen) * dist for (p_gen, dist) in zip(p_gens, attn_dists)]

    # Concatenate some zeros to each vocabulary dist, to hold the probabilities for in-article OOV words
    extended_vsize = vocab_size + batch_oov_len  # the maximum (over the batch) size of the extended vocabulary
    extra_zeros = tf.zeros((batch_size, batch_oov_len))
    # list length max_dec_steps of shape (batch_size, extended_vsize)
    vocab_dists_extended = [tf.concat(axis=1, values=[dist, extra_zeros]) for dist in vocab_dists]

    # Project the values in the attention distributions onto the appropriate entries in the final distributions
    # This means that if a_i = 0.1 and the ith encoder word is w, and w has index 500 in the vocabulary,
    # then we add 0.1 onto the 500th entry of the final distribution
    # This is done for each decoder timestep.
    # This is fiddly; we use tf.scatter_nd to do the projection
    batch_nums = tf.range(0, limit=batch_size)  # shape (batch_size)
    batch_nums = tf.expand_dims(batch_nums, 1)  # shape (batch_size, 1)
    attn_len = tf.shape(_enc_batch_extend_vocab)[1]  # number of states we attend over
    batch_nums = tf.tile(batch_nums, [1, attn_len])  # shape (batch_size, attn_len)
    # shape (batch_size, enc_t, 2)
    indices = tf.stack((batch_nums, _enc_batch_extend_vocab), axis=2)  
    shape = [batch_size, extended_vsize]
    
    # list length max_dec_steps (batch_size, extended_vsize) extended_vsize = 30000 + 2
    attn_dists_projected = [tf.scatter_nd(indices, copy_dist, shape) for copy_dist in attn_dists]

    # Add the vocab distributions and the copy distributions together to get the final distributions
    # final_dists is a list length max_dec_steps; each entry is a tensor shape (batch_size, extended_vsize) giving
    # the final distribution for that decoder timestep
    # Note that for decoder timesteps and examples corresponding to a [PAD] token, this is junk - ignore.
    final_dists = [vocab_dist + copy_dist for (vocab_dist, copy_dist) in
                   zip(vocab_dists_extended, attn_dists_projected)]

    return final_dists

### Q2 .模型运行的时候 上一步预测出来的词 超出vocab范围,下一步输入会不会出问题?

## 1. 训练

In [ ]:
# using teacher forcing
dec_input = tf.expand_dims(dec_target[:, t], 1)

## 2. 测试

In [ ]:
# 替换掉 oov token unknown token
latest_tokens = [t if t in vocab.id2word else unk_index for t in latest_tokens]  

## Q3 Seq2Seq和Point分别起到什么作用?

![](readme/pgn.png)

# 2. coverage

1. attention
2. loss

## Bahdanau Attention

![](img/attention.png)

In [27]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_s = tf.keras.layers.Dense(units)
        self.W_h = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_hidden, enc_output):
        # query为上次的GRU隐藏层
        # values为编码器的编码结果enc_output
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis)))
       
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector,attention_weights

## Coverage Attention

![](img/attention.png)

![](img/e_t.png)

## 改造$e^t$

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_s = tf.keras.layers.Dense(units)
        self.W_h = tf.keras.layers.Dense(units)
        self.W_c = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_hidden, enc_output, enc_pad_mask, use_coverage, prev_coverage):
        # query 隐藏层
        # values为 编码器的编码结果enc_output
        hidden_with_time_axis = tf.expand_dims(dec_hidden, 1)
        # self.W_s(values)  [batch_sz, max_len, units] self.W_h(hidden_with_time_axis) [batch_sz, 1, units]
        # self.W_c(prev_coverage) [batch_sz, max_len, units]  score [batch_sz, max_len, 1]    
        score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis) + self.W_c(prev_coverage)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        # [batch_sz, max_len, enc_units]
        context_vector = attention_weights * enc_output
        # [batch_sz, enc_units]
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector,attention_weights

## mask + coverage

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_s = tf.keras.layers.Dense(units)
        self.W_h = tf.keras.layers.Dense(units)
        self.W_c = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_hidden, enc_output, enc_pad_mask, use_coverage, prev_coverage):
        # query为上次的GRU隐藏层
        # values为编码器的编码结果enc_output
        # 在seq2seq模型中，St是后面的query向量，而编码过程的隐藏状态hi是values。

        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(dec_hidden, 1)

        if use_coverage and prev_coverage is not None:
            # self.W_s(values) [batch_sz, max_len, units] self.W_h(hidden_with_time_axis) [batch_sz, 1, units]
            # self.W_c(prev_coverage) [batch_sz, max_len, units]  score [batch_sz, max_len, 1]
            score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis) + self.W_c(prev_coverage)))
            # attention_weights shape (batch_size, max_len, 1)

            mask = tf.cast(enc_pad_mask, dtype=score.dtype)
            masked_score = tf.squeeze(score, axis=-1) * mask
            masked_score = tf.expand_dims(masked_score, axis=2)

            attention_weights = tf.nn.softmax(masked_score, axis=1)
            coverage = attention_weights + prev_coverage
        else:
            # score shape == (batch_size, max_length, 1)
            # we get 1 at the last axis because we are applying score to self.V
            # the shape of the tensor before applying self.V is (batch_size, max_length, units)
            # 计算注意力权重值
            score = self.V(tf.nn.tanh(
                self.W_s(enc_output) + self.W_h(hidden_with_time_axis)))

            mask = tf.cast(enc_pad_mask, dtype=score.dtype)
            masked_score = tf.squeeze(score, axis=-1) * mask
            masked_score = tf.expand_dims(masked_score, axis=2)

            attention_weights = tf.nn.softmax(masked_score, axis=1)
            # attention_weights = masked_attention(attention_weights)
            if use_coverage:
                coverage = attention_weights

        # attention_weights sha== (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # # 使用注意力权重*编码器输出作为返回值，将来会作为解码器的输入
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector,attention_weights, coverage

## 2.2 coverage_loss

![](img/loss_t.png)

## log loss

In [34]:
# 定义损失函数
def loss_function(real, pred):
    pad_mask = tf.math.equal(real, pad_index)
    mask = tf.math.logical_not(pad_mask)
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

## log loss + mask batch loss

In [35]:
# 定义损失函数
def loss_function(real, pred, padding_mask):
    loss = 0
    for t in range(real.shape[1]):
        if padding_mask:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            mask = tf.cast(padding_mask[:, t], dtype=loss_.dtype)
            loss_ *= mask
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
        else:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
    return tf.reduce_mean(loss)

就是将先前时间步的注意力权重加到一起得到所谓的覆盖向量 $c_t$ (coverage vector)，用先前的注意力权重决策来影响当前注意力权重的决策，这样就避免在同一位置重复，从而避免重复生成文本。计算上，先计算coverage vector $c_t$
![](img/c_t.png)
+ $c^t$就是一个长度为输入长度的向量
+ 第一项是之前时刻输入第一个词attention权重的叠加和
+ 加这个参数的目的是为了给attention之前生成词的信息，如果之前生成过这些词那么后续要抑制。抑制通过loss函数加惩罚项实现.

## 两个地方使用$c_t$:

+ 注意力权重的计算过程中 $e^t_i$
+ cov_loss

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        pass
        
    def call(self, dec_hidden, enc_output, enc_pad_mask, use_coverage, prev_coverage):
        if use_coverage and prev_coverage is not None:
            pass
            attention_weights = tf.nn.softmax(score, axis=1)
            coverage = attention_weights + prev_coverage
        else:    
            if use_coverage:
                coverage = attention_weights

![](img/cobloss.png)

`<START> 举起 车辆 左 前轮 缸体 上 <STOP> <PAD> <PAD> `

`padding_mask`->`[1,1,1,1,1,1,1,0,0]`

In [28]:
def mask_coverage_loss(attn_dists, coverages, padding_mask):
    """
    Calculates the coverage loss from the attention distributions.
      Args:
        attn_dists coverages: [max_len_y, batch_sz, max_len_x, 1]
        padding_mask: shape (batch_size, max_len_y).
      Returns:
        coverage_loss: scalar
    """
    cover_losses = []
    # transfer attn_dists coverages to [max_len_y, batch_sz, max_len_x]
    attn_dists = tf.squeeze(attn_dists, axis=3)
    coverages = tf.squeeze(coverages, axis=3)

    
    for t in range(attn_dists.shape[0]):
        cover_loss_ = tf.reduce_sum(tf.minimum(attn_dists[t, :, :], coverages[t, :, :]), axis=-1)  # max_len_x wise
        cover_losses.append(cover_loss_)
    
    # change from[max_len_y, batch_sz] to [batch_sz, max_len_y]
    cover_losses = tf.stack(cover_losses, 1)

    # cover_loss_ [batch_sz, max_len_y]
    mask = tf.cast(padding_mask, dtype=cover_loss_.dtype)
    cover_losses *= mask
    
    # mean loss of each time step and then sum up
    loss = tf.reduce_sum(tf.reduce_mean(cover_losses, axis=0))  
    tf.print('coverage loss(batch sum):', loss)
    return loss

# loss改变

![](img/loss_t_coverage.png)

In [38]:
batch_loss = loss_function(dec_target[:, 1:], predictions)

In [39]:
batch_loss = loss_function(dec_target, predictions, padding_mask) + \
                         cov_loss_wt * coverage_loss(attentions, coverages, padding_mask)

定义个coverage loss来多次惩罚对相对位置的关注.原理很直观，如果之前该词出现过了，那么它的$c^t_i$就很大，那么为了减少$loss$，就需要$a^t_i$变小（因为loss是取两者较小值）,$a^t_i$小就代表着这个位置被注意的概率减少。

# tensorflow 操作

### tf.constant 操作

In [15]:
x1=tf.constant([[1,1,5],
                [1,1,1]])

x2=tf.constant([[1,3,2],
                [3,1,3]])
tf.minimum(x1, x2)

<tf.Tensor: id=37, shape=(2, 3), dtype=int32, numpy=
array([[1, 1, 2],
       [1, 1, 1]], dtype=int32)>

## tf.reduce_sum 操作

In [19]:
x=tf.constant([[1,1,1],[1,1,1]])
tf.reduce_sum(x)

<tf.Tensor: id=62, shape=(), dtype=int32, numpy=6>

In [20]:
tf.reduce_sum(x,0)

<tf.Tensor: id=64, shape=(3,), dtype=int32, numpy=array([2, 2, 2], dtype=int32)>

In [21]:
tf.reduce_sum(x,1)

<tf.Tensor: id=66, shape=(2,), dtype=int32, numpy=array([3, 3], dtype=int32)>